In [1]:
%load_ext autoreload
%autoreload 2
import warnings 
warnings.filterwarnings("ignore")

In [2]:
from qiskit_metal import designs, MetalGUI
from qiskit_metal.qlibrary.sample_shapes.rectangle import Rectangle
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.Transmon_Interdigitated import TransmonInterdigitated
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.designs.design_multiplanar import MultiPlanar
import pandas as pd


# Main Design

In [3]:
design = MultiPlanar({},overwrite_enabled = True, layer_stack_filename = "layer_stack.txt")

gui = MetalGUI(design)

06:51PM 09s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: Could not create pixmap from c:\usershris\qiskit-metal\qiskit_metal\_gui\styles\metal_dark\rc\transparent.png

06:51PM 09s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: Could not create pixmap from c:\usershris\qiskit-metal\qiskit_metal\_gui\styles\metal_dark\rc\transparent.png



## Qubit Design

In [4]:
design.delete_all_components()

#### Design so Resonator is coupled strongly to each qubit-> large cap leads to large dispersive shift

conn_pads1 = dict(connection_pads = dict(coupler = dict(loc_W=1, loc_H=1, pad_width = '100um'),
                                         readout = dict(loc_W=-1, loc_H=-1, pad_width = '100um')))

conn_pads2 = dict(pad_width = '550um',
                  connection_pads = dict(coupler = dict(loc_W = -1, loc_H = 1,pad_width = '175um'),
                                         readout = dict(loc_W=1, loc_H=-1, pad_width='175um')))
q1 = TransmonPocket(design, "Q1", options=dict(pos_x='-0.5mm', pos_y='+0.0mm', layer=1, **conn_pads1))
q2 = TransmonPocket(design, "Q2", options=dict(pos_x='+0.5mm', pos_y='+0.0mm', layer=1, **conn_pads2))

gui.rebuild()
gui.autoscale()

## Coupler Design

In [5]:
coupler_options = dict(pin_inputs = dict(start_pin = dict(component=q2.name, pin='coupler'),
                                         end_pin = dict(component=q1.name, pin='coupler')))

coupler_connector = RouteStraight(design, 'coupler_connector', options = coupler_options)

gui.rebuild()
gui.autoscale()

## Readout Resonator Design

In [6]:
otg11_options = dict(pos_x='-1.5mm', pos_y = '0.0mm', orientation='180', layer='3')
otg21_options = dict(pos_x='+1.5mm', pos_y = '0.0mm', orientation='0', layer='3')
otg12_options = dict(pos_x='-0.955mm', pos_y = '-0.195mm', orientation='0', layer='3')
otg22_options = dict(pos_x='+0.955mm', pos_y = '-0.1955mm', orientation='180', layer='3')
otg11 = OpenToGround(design, 'otg11' ,options=otg11_options)
otg21 = OpenToGround(design, 'otg21' ,options=otg21_options)
otg12 = OpenToGround(design, 'otg12' ,options=otg12_options)
otg22 = OpenToGround(design, 'otg22' ,options=otg22_options)

readout1_options = dict(total_length = '5.97mm', fillet = '40um', 
                        pin_inputs = dict(start_pin = dict(component = otg11.name, pin='open'),
                                          end_pin = dict(component = otg12.name, pin = 'open')),
                        lead = dict(start_straight='150um'),
                        meander=dict(spacing = '100um', asymmetry = '0'),
                        layer = '3')

readout2_options = dict(total_length = '5.97mm', fillet = '40um', 
                        pin_inputs = dict(start_pin = dict(component = otg22.name, pin='open'),
                                          end_pin = dict(component = otg21.name, pin = 'open')),
                        lead = dict(start_straight='120um'),
                        meander=dict(spacing = '100um', asymmetry = '0.2'),
                        layer = '3')

readout1 = RouteMeander(design, 'readout1', options = readout1_options)
readout2 = RouteMeander(design, 'readout2', options = readout2_options)

gui.rebuild()
gui.autoscale()

## Via Creation and Dielectric

In [7]:
via_positions = [('-0.94mm','-0.1950mm'),('+0.94mm','-0.1950mm')]

for layer in ['1','2','3']:
    for via, positions in enumerate(via_positions):
        for subtract in zip([False,True],['30um','42um']):
            via_size = '20um' if layer =='2' else subtract[1]
            actual_layer = '5' if (layer == '2' and subtract[0]) else layer
            via_options = dict(width = via_size,
                               height = via_size,
                               pos_x = positions[0],
                               pos_y = positions[1],
                               subtract = subtract[0],
                               layer = actual_layer,
                               orientation = '0',
                               helper = 'False',
                               chip = 'main' )
            name = 'via' +str(via+1) +'_layer' + actual_layer+('' if not subtract[0] else '_sub')
            Rectangle(design, name, via_options)
            #print(subtract)
            #print(layer == '2' and subtract[0])
    
gui.rebuild()
gui.autoscale()
            

# Render and Extract Capacitance Matrix

In [8]:
from qiskit_metal.renderers.renderer_elmer.elmer_renderer import QElmerRenderer

## Render

In [9]:
#elmer_renderer = QElmerRenderer(design, layer_types = dict(metal = [1,2,3], dielectric = [4,5]))
#elmer_renderer.render_design(mesh_geoms = True,
#                             skip_junctions= True,
#                             open_pins=[("Q1","readout"),("Q2","readout")],
#                             omit_ground_for_layers = [2])
#elmer_renderer.launch_gmsh_gui()

## Generate Capacitance Matrix

In [10]:
#elmer_renderer.export_mesh()
#elmer_renderer.add_solution_setup('capacitance')
#elmer_renderer.run('capacitance')

In [11]:
#cap_matrix = elmer_renderer.capacitance_matrix
#cap_matrix

In [12]:
#elmer_renderer.display_post_processing_data()
#elmer_renderer.close()

In [13]:
elmer_renderer = QElmerRenderer(design, layer_types=dict(metal=[1,2,3], dielectric=[4,5]))
select = ['Q1','via1_layer1', 'via1_layer2', 'via1_layer3', 'via1_layer1_sub', 'via1_layer5_sub', 'via1_layer3_sub']
#select = ['Q1']
elmer_renderer.render_design(selection=select,  open_pins=[("Q1","readout"),("Q1","coupler")], skip_junctions=True, 
                            omit_ground_for_layers = [2])

#elmer_renderer.launch_gmsh_gui()

In [14]:
elmer_renderer.export_mesh()
elmer_renderer.add_solution_setup('capacitance')
elmer_renderer.run('capacitance')
cap_matrix_q1 = elmer_renderer.capacitance_matrix
cap_matrix_q1.to_csv('cap_matrix_q1.csv')
cap_matrix_q1


06:51PM 37s INFO [run]: Running ElmerGrid on input mesh from Gmsh...
06:51PM 38s INFO [run]: Running ElmerSolver for solver type: 'capacitance'


,Q1_coupler_connector_pad,via1_layer3_rectangle,Q1_pad_top,Q1_pad_bot,ground_plane
Q1_coupler_connector_pad,114.554225,-0.060381,-19.961365,-1.540458,-92.992022
via1_layer3_rectangle,-0.060381,166.363976,-1.553853,-20.094186,-144.655556
Q1_pad_top,-19.961365,-1.553853,237.363976,-52.081053,-163.767705
Q1_pad_bot,-1.540458,-20.094186,-52.081053,237.288838,-163.573141
ground_plane,-92.992022,-144.655556,-163.767705,-163.573141,300.000000


In [15]:
elmer_renderer = QElmerRenderer(design, layer_types=dict(metal=[1,2,3], dielectric=[4,5]))
select = ['Q2','via2_layer1', 'via2_layer2', 'via2_layer3', 'via2_layer1_sub', 'via2_layer5_sub', 'via2_layer3_sub']
#select = ['Q2']
elmer_renderer.render_design(selection=select,  open_pins=[("Q2","readout"),("Q2","coupler")], skip_junctions=True, 
                            omit_ground_for_layers = [2])

#elmer_renderer.launch_gmsh_gui()

In [16]:
elmer_renderer.export_mesh()
elmer_renderer.add_solution_setup('capacitance')
elmer_renderer.run('capacitance')
cap_matrix_q2 = elmer_renderer.capacitance_matrix
cap_matrix_q2.to_csv('cap_matrix_q2.csv')
cap_matrix_q2


06:52PM 13s INFO [run]: Running ElmerGrid on input mesh from Gmsh...
06:52PM 14s INFO [run]: Running ElmerSolver for solver type: 'capacitance'


,Q2_coupler_connector_pad,via2_layer3_rectangle,Q2_pad_top,Q2_pad_bot,ground_plane
Q2_coupler_connector_pad,136.609610,-0.085335,-30.673695,-1.892989,-103.957592
via2_layer3_rectangle,-0.085335,187.795334,-1.924359,-30.474603,-155.311036
Q2_pad_top,-30.673695,-1.924359,288.164636,-61.627467,-193.939115
Q2_pad_bot,-1.892989,-30.474603,-61.627467,287.823071,-193.828011
ground_plane,-103.957592,-155.311036,-193.939115,-193.828011,300.000000


,Q2_coupler_connector_pad,via2_layer3_rectangle,Q2_pad_top,Q2_pad_bot,ground_plane
Q2_coupler_connector_pad,168.277993,-0.270188,-45.617732,-2.710089,-119.679984
via2_layer3_rectangle,-0.270188,220.584424,-2.732696,-45.566898,-172.014641
Q2_pad_top,-45.617732,-2.732696,296.128120,-60.479065,-187.298627
Q2_pad_bot,-2.710089,-45.566898,-60.479065,296.211562,-187.455510
ground_plane,-119.679984,-172.014641,-187.298627,-187.455510,300.000000


# LOM Analysis

In [17]:
import scqubits as scq
from scipy.constants import speed_of_light as c_light
import matplotlib.pyplot as plt
from qiskit_metal.analyses.quantization.lumped_capacitive import load_q3d_capacitance_matrix
from qiskit_metal.analyses.quantization.lom_core_analysis import CompositeSystem, Cell, Subsystem, QuantumSystemRegistry
import pandas as pd
cap_matrix_q1 = pd.read_csv('cap_matrix_q1.csv').set_index('Unnamed: 0',True)
cap_matrix_q1.index.name = None

cap_matrix_q2 = pd.read_csv('cap_matrix_q2.csv').set_index('Unnamed: 0',True)
cap_matrix_q2.index.name = None


06:52PM 39s INFO [__init__]: TransmonBuilder with system_type TRANSMON registered to QuantumSystemRegistry
06:52PM 39s INFO [__init__]: FluxoniumBuilder with system_type FLUXONIUM registered to QuantumSystemRegistry
06:52PM 39s INFO [__init__]: TLResonatorBuilder with system_type TL_RESONATOR registered to QuantumSystemRegistry
06:52PM 39s INFO [__init__]: LumpedResonatorBuilder with system_type LUMPED_RESONATOR registered to QuantumSystemRegistry


## LOM Cells

In [18]:
opt1 = dict(node_rename = {'Q1_coupler_connector_pad':'coupling','via1_layer3_rectangle': 'readout_alice'},
#opt1 = dict(node_rename = {'Q1_coupler_connector_pad':'coupling','Q1_readout_connector_pad': 'readout_alice'},
            cap_mat = cap_matrix_q1,
            ind_dict = {('Q1_pad_top','Q1_pad_bot'):10},
            jj_dict = {('Q1_pad_top','Q1_pad_bot'):'j1'},
            cj_dict = {('Q1_pad_top','Q1_pad_bot'):2}
           )            

cell_1 = Cell(opt1)

opt2 = dict(node_rename = {'Q2_coupler_connector_pad':'coupling','via2_layer3_rectangle': 'readout_bob'},
#opt2 = dict(node_rename = {'Q2_coupler_connector_pad':'coupling','Q2_readout_connector_pad': 'readout_bob'},
            cap_mat = cap_matrix_q2,
            ind_dict = {('Q2_pad_top','Q2_pad_bot'):12},
            jj_dict = {('Q2_pad_top','Q2_pad_bot'):'j2'},
            cj_dict = {('Q2_pad_top','Q2_pad_bot'):2}
           )
cell_2 = Cell(opt2)

## Subsystems

In [19]:
transmon_alice = Subsystem(name = 'transmon_alice', sys_type='TRANSMON', nodes=['j1'])

transmon_bob = Subsystem(name = 'transmon_bob', sys_type='TRANSMON', nodes=['j2'])


### Readout resonator properties

#Alice
q_opts1 = dict(
    f_res = 5.6, ##in GHz
    Z0 = 50,  #charicterisit impedance
    vp = 0.40314 * c_light, ## phase velocity
)
res_alice = Subsystem(name='readout_alice', sys_type ='TL_RESONATOR', nodes = ['readout_alice'], q_opts = q_opts1)


#Bob
q_opts2 = dict(
    f_res = 4.0, ##in GHz
    Z0 = 50,  #charicterisit impedance
    vp = 0.40314 * c_light, ## phase velocity
)
res_bob = Subsystem(name='readout_bob', sys_type ='TL_RESONATOR', nodes = ['readout_bob'], q_opts = q_opts2)






## Create Composite System

In [20]:
composite_sys = CompositeSystem(
            subsystems = [transmon_alice, transmon_bob, res_alice, res_bob],
            cells = [cell_1, cell_2],
            grd_node = 'ground_plane', #ID Ground plane to remove
            nodes_force_keep = ['readout_alice', 'readout_bob'] #need to do this so hamiltonian includes readout res's
            )

In [21]:
cg = composite_sys.circuitGraph()
print(cg)

node_jj_basis:
-------------

['j1', 'Q1_pad_bot', 'j2', 'Q2_pad_bot', 'readout_alice', 'readout_bob', 'coupling']

nodes_keep:
-------------

['j1', 'j2', 'readout_alice', 'readout_bob']


L_inv_k (reduced inverse inductance matrix):
-------------

                j1        j2  readout_alice  readout_bob
j1             0.1  0.000000            0.0          0.0
j2             0.0  0.083333            0.0          0.0
readout_alice  0.0  0.000000            0.0          0.0
readout_bob    0.0  0.000000            0.0          0.0

C_k (reduced capacitance matrix):
-------------

                       j1          j2  readout_alice  readout_bob
j1             146.361233   -0.534780       9.223387    -0.089858
j2              -0.534780  175.975575      -0.076471    14.147037
readout_alice    9.223387   -0.076471     165.092062    -0.012849
readout_bob     -0.089858   14.147037      -0.012849   185.453189




In [22]:
## Creats hilbert space for seqeuncing analysis
hilbertspace = composite_sys.create_hilbertspace()
hilbertspace = composite_sys.add_interaction()
hilbertspace.hamiltonian()

Quantum object: dims = [[10, 10, 3, 3], [10, 10, 3, 3]], shape = (900, 900), type = oper, isherm = True
Qobj data =
[[-26209.06108658+0.j              0.        +0.03889068j
       0.        +0.j         ...      0.        +0.j
       0.        +0.j              0.        +0.j        ]
 [     0.        -0.03889068j -22209.06108658+0.j
       0.        +0.05499972j ...      0.        +0.j
       0.        +0.j              0.        +0.j        ]
 [     0.        +0.j              0.        -0.05499972j
  -18209.06108658+0.j         ...      0.        +0.j
       0.        +0.j              0.        +0.j        ]
 ...
 [     0.        +0.j              0.        +0.j
       0.        +0.j         ...  40060.3322037 +0.j
       0.        +0.78028012j      0.        +0.j        ]
 [     0.        +0.j              0.        +0.j
       0.        +0.j         ...      0.        -0.78028012j
   44060.3322037 +0.j              0.        +1.10348273j]
 [     0.        +0.j              0.   

In [23]:
print(hilbertspace)

HilbertSpace:  subsystems
-------------------------

Transmon------------| [Transmon_3]
                    | EJ: 16346.15128067812
                    | EC: 132.81441642488147
                    | ng: 0.001
                    | ncut: 22
                    | truncated_dim: 10
                    |
                    | dim: 45


Transmon------------| [Transmon_4]
                    | EJ: 13621.792733898432
                    | EC: 110.75381052107379
                    | ng: 0.001
                    | ncut: 22
                    | truncated_dim: 10
                    |
                    | dim: 45


Oscillator----------| [Oscillator_3]
                    | E_osc: 5600.0
                    | l_osc: None
                    | truncated_dim: 3
                    |
                    | dim: 3


Oscillator----------| [Oscillator_4]
                    | E_osc: 4000.0
                    | l_osc: None
                    | truncated_dim: 3
                    |
                 

In [24]:
hamiltonian_results = composite_sys.hamiltonian_results(hilbertspace) # anharminicity are the diagonal in MHz
#off diagonal show dispersive shifts


system frequencies in GHz:
--------------------------
{'transmon_alice': 4.02760507963676, 'transmon_bob': 3.3545556403177472, 'readout_alice': 5.601805595624101, 'readout_bob': 4.004085886141642}

Chi matrices in MHz
--------------------------
                transmon_alice  transmon_bob  readout_alice  readout_bob
transmon_alice     -143.474570     -0.045976      -0.484912     0.006092
transmon_bob         -0.045976   -118.392614      -0.000030    -1.621602
readout_alice        -0.484912     -0.000030       1.052130    -0.000053
readout_bob           0.006092     -1.621602      -0.000053     1.150187


In [25]:
transmon_alice.h_params

defaultdict(dict,
            {'j1': {'EJ': 16346.15128067812,
              'EC': 132.81441642488147,
              'Q_zpf': 3.204353268e-19,
              'default_charge_op': Operator(op=array([[-22,   0,   0, ...,   0,   0,   0],
                     [  0, -21,   0, ...,   0,   0,   0],
                     [  0,   0, -20, ...,   0,   0,   0],
                     ...,
                     [  0,   0,   0, ...,  20,   0,   0],
                     [  0,   0,   0, ...,   0,  21,   0],
                     [  0,   0,   0, ...,   0,   0,  22]]), add_hc=False)}})

In [26]:
transmon_bob.h_params

defaultdict(dict,
            {'j2': {'EJ': 13621.792733898432,
              'EC': 110.75381052107379,
              'Q_zpf': 3.204353268e-19,
              'default_charge_op': Operator(op=array([[-22,   0,   0, ...,   0,   0,   0],
                     [  0, -21,   0, ...,   0,   0,   0],
                     [  0,   0, -20, ...,   0,   0,   0],
                     ...,
                     [  0,   0,   0, ...,  20,   0,   0],
                     [  0,   0,   0, ...,   0,  21,   0],
                     [  0,   0,   0, ...,   0,   0,  22]]), add_hc=False)}})

# Time Evolution Simulation (Sequencing)

## Start here if kernal crashes

In [27]:
import scqubits as scq
from scipy.constants import speed_of_light as c_light
import matplotlib.pyplot as plt
from qiskit_metal.analyses.quantization.lumped_capacitive import load_q3d_capacitance_matrix
from qiskit_metal.analyses.quantization.lom_core_analysis import CompositeSystem, Cell, Subsystem, QuantumSystemRegistry
import pandas as pd
cap_matrix_q1 = pd.read_csv('cap_matrix_q1.csv').set_index('Unnamed: 0',True)
cap_matrix_q1.index.name = None

cap_matrix_q2 = pd.read_csv('cap_matrix_q2.csv').set_index('Unnamed: 0',True)
cap_matrix_q2.index.name = None

opt1 = dict(node_rename = {'Q1_coupler_connector_pad':'coupling','via1_layer3_rectangle': 'readout_alice'},
#opt1 = dict(node_rename = {'Q1_coupler_connector_pad':'coupling','Q1_readout_connector_pad': 'readout_alice'},
            cap_mat = cap_matrix_q1,
            ind_dict = {('Q1_pad_top','Q1_pad_bot'):10},
            jj_dict = {('Q1_pad_top','Q1_pad_bot'):'j1'},
            cj_dict = {('Q1_pad_top','Q1_pad_bot'):2}
           )            

cell_1 = Cell(opt1)

opt2 = dict(node_rename = {'Q2_coupler_connector_pad':'coupling','via2_layer3_rectangle': 'readout_bob'},
#opt2 = dict(node_rename = {'Q2_coupler_connector_pad':'coupling','Q2_readout_connector_pad': 'readout_bob'},
            cap_mat = cap_matrix_q2,
            ind_dict = {('Q2_pad_top','Q2_pad_bot'):12},
            jj_dict = {('Q2_pad_top','Q2_pad_bot'):'j2'},
            cj_dict = {('Q2_pad_top','Q2_pad_bot'):2}
           )
cell_2 = Cell(opt2)

transmon_alice = Subsystem(name = 'transmon_alice', sys_type='TRANSMON', nodes=['j1'])

transmon_bob = Subsystem(name = 'transmon_bob', sys_type='TRANSMON', nodes=['j2'])


### Readout resonator properties

#Alice
q_opts1 = dict(
    f_res = 5.6, ##in GHz
    Z0 = 50,  #charicterisit impedance
    vp = 0.40314 * c_light, ## phase velocity
)
res_alice = Subsystem(name='readout_alice', sys_type ='TL_RESONATOR', nodes = ['readout_alice'], q_opts = q_opts1)


#Bob
q_opts2 = dict(
    f_res = 4.0, ##in GHz
    Z0 = 50,  #charicterisit impedance
    vp = 0.40314 * c_light, ## phase velocity
)
res_bob = Subsystem(name='readout_bob', sys_type ='TL_RESONATOR', nodes = ['readout_bob'], q_opts = q_opts2)



composite_sys = CompositeSystem(
            subsystems = [transmon_alice, transmon_bob, res_alice, res_bob],
            cells = [cell_1, cell_2],
            grd_node = 'ground_plane', #ID Ground plane to remove
            nodes_force_keep = ['readout_alice', 'readout_bob'] #need to do this so hamiltonian includes readout res's
            )

## Creats hilbert space for seqeuncing analysis
hilbertspace = composite_sys.create_hilbertspace()
hilbertspace = composite_sys.add_interaction()
hilbertspace.hamiltonian()

Quantum object: dims = [[10, 10, 3, 3], [10, 10, 3, 3]], shape = (900, 900), type = oper, isherm = True
Qobj data =
[[-26209.06108658+0.j              0.        +0.03889068j
       0.        +0.j         ...      0.        +0.j
       0.        +0.j              0.        +0.j        ]
 [     0.        -0.03889068j -22209.06108658+0.j
       0.        +0.05499972j ...      0.        +0.j
       0.        +0.j              0.        +0.j        ]
 [     0.        +0.j              0.        -0.05499972j
  -18209.06108658+0.j         ...      0.        +0.j
       0.        +0.j              0.        +0.j        ]
 ...
 [     0.        +0.j              0.        +0.j
       0.        +0.j         ...  40060.3322037 +0.j
       0.        +0.78028012j      0.        +0.j        ]
 [     0.        +0.j              0.        +0.j
       0.        +0.j         ...      0.        -0.78028012j
   44060.3322037 +0.j              0.        +1.10348273j]
 [     0.        +0.j              0.   

In [28]:
from tqdm import tqdm
from functools import partial
import qutip

from sequencing import Qubit, Transmon, Cavity, System, get_sequence, sync, delay, QasmSequence
from sequencing.calibration import fit_sine, fit_displacement, tune_rabi, tune_displacement, tune_drag

from qiskit_metal.analyses.quantization.lom_time_evolution_sim import lom_composite_sys_to_seq_sys

In [29]:
system = lom_composite_sys_to_seq_sys(composite_sys, hilbertspace, levels = [3,3,10,10])


In [30]:
system.modes #Mode is a subsystem in LOM analysis

[Transmon(name='transmon_bob', cls='sequencing.modes.Transmon', levels=3, t1=inf, t2=inf, thermal_population=0.0, df=0.0, kerr=-0.11839261375115893, pulses={'smoothed_constant_pulse': SmoothedConstantPulse(name='smoothed_constant_pulse', cls='sequencing.pulses.SmoothedConstantPulse', amp=1.0, detune=0.0, phase=0.0, dt=1, noise_sigma=0.0, noise_alpha=0.0, scale_noise=False, length=100, sigma=0, shape='tanh'), 'gaussian_pulse': GaussianPulse(name='gaussian_pulse', cls='sequencing.pulses.GaussianPulse', amp=1.0, detune=0.0, phase=0.0, dt=1, noise_sigma=0.0, noise_alpha=0.0, scale_noise=False, sigma=10, chop=4, drag=0.0)}, default_pulse='gaussian_pulse'),
 Transmon(name='transmon_alice', cls='sequencing.modes.Transmon', levels=3, t1=inf, t2=inf, thermal_population=0.0, df=0.0, kerr=-0.1434745702223954, pulses={'smoothed_constant_pulse': SmoothedConstantPulse(name='smoothed_constant_pulse', cls='sequencing.pulses.SmoothedConstantPulse', amp=1.0, detune=0.0, phase=0.0, dt=1, noise_sigma=0.0,

## Simulate Alice

In [31]:
alice = system.modes[1]
readout_alice = system.modes[3]
print(alice)
print(readout_alice)

Transmon(name='transmon_alice', cls='sequencing.modes.Transmon', levels=3, t1=inf, t2=inf, thermal_population=0.0, df=0.0, kerr=-0.1434745702223954, pulses={'smoothed_constant_pulse': SmoothedConstantPulse(name='smoothed_constant_pulse', cls='sequencing.pulses.SmoothedConstantPulse', amp=1.0, detune=0.0, phase=0.0, dt=1, noise_sigma=0.0, noise_alpha=0.0, scale_noise=False, length=100, sigma=0, shape='tanh'), 'gaussian_pulse': GaussianPulse(name='gaussian_pulse', cls='sequencing.pulses.GaussianPulse', amp=1.0, detune=0.0, phase=0.0, dt=1, noise_sigma=0.0, noise_alpha=0.0, scale_noise=False, sigma=10, chop=4, drag=0.0)}, default_pulse='gaussian_pulse')
Cavity(name='readout_alice', cls='sequencing.modes.Cavity', levels=10, t1=inf, t2=inf, thermal_population=0.0, df=0.0, kerr=0.0010521299837291736, pulses={'smoothed_constant_pulse': SmoothedConstantPulse(name='smoothed_constant_pulse', cls='sequencing.pulses.SmoothedConstantPulse', amp=1.0, detune=0.0, phase=0.0, dt=1, noise_sigma=0.0, noi

In [50]:
selective_sigma = 100 #ns

with system.use_modes([alice]):
        with alice.temporarily_set(gaussian_pulse__sigma = selective_sigma):
            _, _, selective_qubit_amp = tune_rabi(
            system, system.fock(transmon_alice = 0, transmon_bob = 0, readout_alice =0, readout_bob=0), mode_name = alice.name,
            update = False, verify=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 51/51 [00:00<00:00, 97.08it/s]


In [51]:
def selective_rotation(qubit, angle, phase =0, detune=0, sigma = selective_sigma):
    with qubit.gaussian_pulse.temporarily_set(sigma=sigma, amp  = selective_qubit_amp):
        qubit.rotate(np.pi, phase, detune=detune)

init_states = [
    (f'$|g{n}\\rangle$', system.fock(transmon_alice=0, readout_alice=n)) for n in range(4)
]

In [52]:
import numpy as np
results = {}

seq = get_sequence(system)
selective_rotation(alice, np.pi)

for label, state in tqdm(init_states, desc='Initial states'):
    result = seq.run(state)
    results[label] = result

Initial states: 100%|████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 10.21it/s]


In [53]:
fig, ax = plt.subplots(1,1)
for label, result in results.items():
    qubit_states = [state.ptrace(alice.index) for state in result.states]
    e_pops = qutip.expect(alice.fock_dm(1, full_space=False), qubit_states)
    ax.plot(result.times, e_pops, label=label)
    ax.grid(True)
    ax.legend(loc=0)
ax.set_xlabel('Times [ns]')
ax.set_ylabel(r'$P(|e\rangle)$')
_ = ax.set_title('Transmon trajectory vs. Initital cavity state')

In [36]:
bob = system.modes[0]
readout_bob = system.modes[2]
print(bob)
print(readout_bob)

Transmon(name='transmon_bob', cls='sequencing.modes.Transmon', levels=3, t1=inf, t2=inf, thermal_population=0.0, df=0.0, kerr=-0.11839261375115893, pulses={'smoothed_constant_pulse': SmoothedConstantPulse(name='smoothed_constant_pulse', cls='sequencing.pulses.SmoothedConstantPulse', amp=1.0, detune=0.0, phase=0.0, dt=1, noise_sigma=0.0, noise_alpha=0.0, scale_noise=False, length=100, sigma=0, shape='tanh'), 'gaussian_pulse': GaussianPulse(name='gaussian_pulse', cls='sequencing.pulses.GaussianPulse', amp=1.0, detune=0.0, phase=0.0, dt=1, noise_sigma=0.0, noise_alpha=0.0, scale_noise=False, sigma=10, chop=4, drag=0.0)}, default_pulse='gaussian_pulse')
Cavity(name='readout_bob', cls='sequencing.modes.Cavity', levels=10, t1=inf, t2=inf, thermal_population=0.0, df=0.0, kerr=0.0011501872532935521, pulses={'smoothed_constant_pulse': SmoothedConstantPulse(name='smoothed_constant_pulse', cls='sequencing.pulses.SmoothedConstantPulse', amp=1.0, detune=0.0, phase=0.0, dt=1, noise_sigma=0.0, noise_

In [37]:
selective_sigma = 100 #ns

with system.use_modes([bob]):
        with bob.temporarily_set(gaussian_pulse__sigma = selective_sigma):
            _, _, selective_qubit_amp = tune_rabi(
            system, system.fock(transmon_alice = 0, transmon_bob = 0, readout_alice =0, readout_bob=0), mode_name = bob.name,
            update = False, verify=False)

100%|█████████████████████████████████████████████████████████████████████████████████| 51/51 [00:00<00:00, 102.84it/s]


In [38]:
init_states = [
    (f'$|g{n}\\rangle$', system.fock(transmon_bob=0, readout_bob=n)) for n in range(4)
]

In [39]:
import numpy as np
results = {}

seq = get_sequence(system)
selective_rotation(bob, np.pi)

for label, state in tqdm(init_states, desc='Initial states'):
    result = seq.run(state)
    results[label] = result

Initial states: 100%|████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 10.42it/s]


In [40]:
fig, ax = plt.subplots(1,1)
for label, result in results.items():
    qubit_states = [state.ptrace(bob.index) for state in result.states]
    e_pops = qutip.expect(bob.fock_dm(1, full_space=False), qubit_states)
    ax.plot(result.times, e_pops, label=label)
    ax.grid(True)
    ax.legend(loc=0)
ax.set_xlabel('Times [ns]')
ax.set_ylabel(r'$P(|e\rangle)$')
_ = ax.set_title('Transmon trajectory vs. Initital cavity state')

In [35]:
Qubits  = [alice,bob]
qreg = System('qreg',modes = Qubits)
n=len(Qubits)
qreg

System(name='qreg', cls='sequencing.system.System', modes=[Transmon(name='transmon_bob', cls='sequencing.modes.Transmon', levels=3, t1=inf, t2=inf, thermal_population=0.0, df=0.0, kerr=-0.1184731995316688, pulses={'smoothed_constant_pulse': SmoothedConstantPulse(name='smoothed_constant_pulse', cls='sequencing.pulses.SmoothedConstantPulse', amp=1.0, detune=0.0, phase=0.0, dt=1, noise_sigma=0.0, noise_alpha=0.0, scale_noise=False, length=100, sigma=0, shape='tanh'), 'gaussian_pulse': GaussianPulse(name='gaussian_pulse', cls='sequencing.pulses.GaussianPulse', amp=1.0, detune=0.0, phase=0.0, dt=1, noise_sigma=0.0, noise_alpha=0.0, scale_noise=False, sigma=10, chop=4, drag=0.0)}, default_pulse='gaussian_pulse'), Transmon(name='transmon_alice', cls='sequencing.modes.Transmon', levels=3, t1=inf, t2=inf, thermal_population=0.0, df=0.0, kerr=-0.14346319786989625, pulses={'smoothed_constant_pulse': SmoothedConstantPulse(name='smoothed_constant_pulse', cls='sequencing.pulses.SmoothedConstantPulse

In [36]:
for qubit in Qubits:
    with system.use_modes([qubit]):
        with qubit.temporarily_set(gaussian_pulse__sigma = selective_sigma):
            _, _, selective_qubit_amp = tune_rabi(
            system, system.fock(transmon_alice = 0, transmon_bob = 0, readout_alice =0, readout_bob=0), mode_name = qubit.name,
            update = False, verify=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 51/51 [00:00<00:00, 98.22it/s]


In [27]:
QASM_CIRCUIT = [
    f'OPENQASM 2.0;',
    f'include "qelib1.inc";',
    f'qreg q[{n}];',
    f'h q[0];',
    f'barrier;',
]
QASM_CIRCUIT.extend(
    [f'CX q[0],q[{i}];' for i in range(1,n)]
)

QASM_CIRCUIT = '\n\t'.join([''] + QASM_CIRCUIT)

print('Running the following QASM circuit:')
print(QASM_CIRCUIT)

seq = QasmSequence(system)
seq.qasm_circuit(QASM_CIRCUIT, unitary=False, append=True)

Running the following QASM circuit:

	OPENQASM 2.0;
	include "qelib1.inc";
	qreg q[2];
	h q[0];
	barrier;
	CX q[0],q[1];


AttributeError: If unitary is False, all qubits must implement rotate_y (readout_alice does not).

In [102]:
ax = qubit.gaussian_pulse.plot()
_ = ax.set_xlabel('Time [ns]')